In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from numpy.random import uniform, lognormal, normal
import matplotlib.pyplot as plt 
from astropy.modeling.models import Sersic2D


In [2]:
def make_galaxy_parameters(n_images=100000, n_gal_per_image=2,
                         img_size=(64,64), seed = 12345, plot = True):
    n_gals = (n_gal_per_image, n_images)
    np.random.seed(seed)
    params1 = dict(
        amplitude = lognormal(0, 0.1, size=n_gals),
        r_eff = lognormal(2, 0.1, size=n_gals),
        n = lognormal(0.5, 0.2, size=n_gals),
        ellip = uniform(0.0, 0.8, size=n_gals),
        x_0 = normal(0.5, 0.1, size=n_gals) * img_size[0],
        y_0 = normal(0.5, 0.1, size=n_gals) * img_size[1],
        theta = uniform(0.0, np.pi, size=n_gals)
    )
    if plot:
        fig, axarr = plt.subplots(2, 4, figsize=(15, 8))
        for ax, (par, values) in zip(axarr.flat, params1.items()):
            ax.hist(values, histtype='step')
            ax.set_xlabel(par)
        axarr.flat[-1].axis('off')
    return params1

In [3]:
def make_blended_gals(params, img_size=(64, 64)):
    x, y = np.mgrid[:img_size[1], :img_size[0]]
    shape = (2, ngal)
    components = np.zeros(shape + img_size[::-1])
    blended = np.zeros(shape[1:] + img_size[::-1])
    for img_idx in range(shape[1]):
        for gal_idx in range(shape[0]):
            par = {p: x[gal_idx, img_idx] for p, x in params.items()}
            mod = Sersic2D(**par)(y, x)
            components[gal_idx, img_idx] = mod
            blended[img_idx] += mod
    return components, blended

In [4]:
def add_noise(img, scale=0.01):
    return img + normal(scale=scale, size=img.shape)


In [5]:
def asinh_scale(x, a=0.1, inverse=False):
    # This is a common way of scaling astronomical images
    # so that inner and outer regions of galaxies can be seen
    return np.arcsinh(x / a) / np.arcsinh(1 / a)

In [6]:

ngal = 100000
ncomp = 2

params = make_galaxy_parameters(ngal, ncomp, plot=False)

components, blended = make_blended_gals(params)

blended = add_noise(blended)
components = add_noise(components)

components = asinh_scale(components)
blended = asinh_scale(blended)


In [9]:
np.save("blends.npy", blended)
np.save("components.npy", components)